In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2021,19)
from solution import parse_data, Scanner
from grid import manhattan_distance

In [2]:
input_text = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

In [3]:
scanners = parse_data(input_text)
master = scanners[0]
minion = scanners[1]
while any([not scanner.aligned for scanner in scanners]):
    for scanner in scanners:
        if scanner.aligned:
            continue
        for other in scanners:
            if not other.aligned:
                continue
            print(f"Trying to align: {scanner.scanner_id} with {other.scanner_id} ", end='')
            scanner.align(other)
            print(scanner.aligned)
            if scanner.aligned:
                break

all_beacons = set()
for scanner in scanners:
    all_beacons.update(scanner.beacons)
print(len(all_beacons))
max_distance = 0
for scanner in scanners:
    for other in scanners:
        max_distance = max(max_distance, manhattan_distance(scanner.position, other.position))
print(max_distance)

Trying to align: 1 with 0 True
Trying to align: 2 with 0 False
Trying to align: 2 with 1 False
Trying to align: 3 with 0 False
Trying to align: 3 with 1 True
Trying to align: 4 with 0 False
Trying to align: 4 with 1 True
Trying to align: 2 with 0 False
Trying to align: 2 with 1 False
Trying to align: 2 with 3 False
Trying to align: 2 with 4 True
79
3621


In [4]:
int('a')
import re
import numpy as np
from itertools import permutations, product
from collections import defaultdict
from functools import lru_cache

class Scanner():
    pattern_digit = re.compile(r'(\-*\d+)')

    def __init__(self, scanner_text):
        """init"""
        lines = scanner_text.splitlines()
        header = lines.pop(0)
        self.scanner_id = int(self.pattern_digit.findall(header)[0])
        self.beacons = set()
        self.position = None
        self.aligned = False
        self.translation_vector = None
        self.translated = False
        if self.scanner_id == 0:
            self.position = (0, 0, 0)
            self.aligned = True
            self.translated = True
        for line in lines:
            point = [int(num) for num in self.pattern_digit.findall(line)]
            self.beacons.add(tuple(point))
        self.initial_beacons = set(self.beacons)

    def __str__(self):
        """String"""
        my_str = f"scanner_id: {self.scanner_id}, aligned: {self.aligned}"
        my_str += f", position: {self.position}, translation_vector: {self.translation_vector}\n"
        # for beacon in self.beacons:
        #     my_str += f"  {beacon}\n"
        return my_str.rstrip()
    
    def rotate_beacons(self, axis):
        current = list(self.beacons)
        new = []
        for beacon in current:
                new.append(rotate_point(beacon, 'x'))
        self.beacons = set(new)
        # print(f"current: {sorted(current)}")
        # print(f"new: {sorted(new)}")
        # print(f"self.beacons: {sorted(self.beacons)}")

    def align(self, other):
        # print(f"pre align {self.scanner_id}: {sorted(self.beacons)}")
        if self.compare(other) > 11:
            self.aligned = True
            # print(f"post align A {self.scanner_id}: {sorted(self.beacons)}")
            return self.aligned
        for _ in range(4):
            self.rotate_beacons('x')
            if self.compare(other) > 11:
                self.aligned = True
                # print(f"post align B {self.scanner_id}: {sorted(self.beacons)}")
                return self.aligned
            for _ in range(4):
                self.rotate_beacons('y')
                if self.compare(other) > 11:
                    self.aligned = True
                    # print(f"post align C {self.scanner_id}: {sorted(self.beacons)}")
                    return self.aligned
                for _ in range(4):
                    self.rotate_beacons('z')
                    if self.compare(other) > 11:
                        self.aligned = True
                        # print(f"post align D {self.scanner_id}: {sorted(self.beacons)}")
                        return self.aligned
        self.aligned = False
        # print(f"post align E {self.scanner_id}: {sorted(self.beacons)}")
        return self.aligned
    
    def matches(self, other):
        matches = defaultdict(set)
        for point in self.beacons:
            for other_point in other.beacons:
                for rotation in generate_all_rotations(other_point):
                    translation_vector = calculate_translation_vector(point, rotation)
                    matches[translation_vector].add((point, rotation))
                    if len(matches[translation_vector]) == 12:
                        print(f"point: {point}, other_point: {other_point}, rotation: {rotation}, translation_vector: {translation_vector}")
                        self.translation_vector = translation_vector
                        return matches[translation_vector]
        return set()

    def compare(self, other):
        return len(self.matches(other))
    
    def translate(self):
        if self.translated:
            return
        beacons = set()
        for point in self.beacons:
            t_point = []
            for idx, val in enumerate(point):
                t_point.append(val + self.translation_vector[idx])
            beacons.add(tuple(t_point))
        self.beacons = beacons
        self.position = self.translation_vector
        self.translated = True
        return
            
            

@lru_cache(maxsize=None)
def calculate_translation_vector(point, other):
    """Method to calculate a translation vector"""
    return tuple(p - o for p, o in zip(point, other))

def matching_points(points, others):
    matches = defaultdict(set)
    for point in points:
        for other in others:
            for rotation in generate_all_rotations(other):
                translation_vector = calculate_translation_vector(point, rotation)
                matches[translation_vector].add((point, rotation))
                if len(matches[translation_vector]) == 12:
                    print(f"point: {point}, other: {other}, rotation: {rotation}, translation_vector: {translation_vector}")
                    return matches[translation_vector]
    # for translation_vector, pairs in matches.items():
    #     if len(pairs) > 11:
    #         # print(f"translation_vector: {translation_vector}, count: {len(pairs)}")
    #         return pairs
    return set()
    
@lru_cache(maxsize=None)
def rotate_point(point, axis, angle_degrees=90):
    """
    Rotates a point in 3D space around the specified axis by a given angle.
    
    Parameters:
        point (tuple or list): The point (x, y, z) to rotate.
        axis (str): The axis of rotation ('x', 'y', or 'z').
        angle_degrees (float): The angle of rotation in degrees.
    
    Returns:
        tuple: The rotated point (x', y', z').
    """
    # Convert the angle from degrees to radians
    angle_radians = np.radians(angle_degrees)
    
    # Define the rotation matrices for each axis
    if axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(angle_radians), -np.sin(angle_radians)],
            [0, np.sin(angle_radians), np.cos(angle_radians)]
        ])
    elif axis == 'y':
        rotation_matrix = np.array([
            [np.cos(angle_radians), 0, np.sin(angle_radians)],
            [0, 1, 0],
            [-np.sin(angle_radians), 0, np.cos(angle_radians)]
        ])
    elif axis == 'z':
        rotation_matrix = np.array([
            [np.cos(angle_radians), -np.sin(angle_radians), 0],
            [np.sin(angle_radians), np.cos(angle_radians), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'.")

    # Convert the point to a numpy array and apply the rotation
    point_vector = np.array(point)
    rotated_point = np.dot(rotation_matrix, point_vector)
    
    # Return the result as a tuple
    return tuple([round(val) for val in rotated_point])

@lru_cache(maxsize=None)
def generate_all_rotations(point):
    """
    Generate all 24 unique orientations of a 3D point using the cube's full symmetry group.

    Parameters:
        point (tuple): The initial 3D point (x, y, z).

    Returns:
        set: A set of 24 unique rotated points.
    """
    point_vector = np.array(point)
    unique_points = set()

    # Generate the 24 rotation matrices using cube symmetries
    base_axes = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # Standard basis
    signs = [1, -1]

    for permuted_axes in permutations(base_axes):
        for sign_comb in product(signs, repeat=3):
            rotation_matrix = np.array([s * np.array(a) for s, a in zip(sign_comb, permuted_axes)])
            
            # Ensure the rotation matrix is valid (determinant == 1)
            if np.linalg.det(rotation_matrix) == 1:
                rotated_point = tuple(np.dot(rotation_matrix, point_vector))
                unique_points.add(rotated_point)

    return unique_points

def parse_data(text):
    scanners = {}
    scanner_texts = text.split('\n\n')
    for scanner_text in scanner_texts:
        scanner = Scanner(scanner_text)
        scanners[scanner.scanner_id] = scanner
    return scanners


ValueError: invalid literal for int() with base 10: 'a'

In [ ]:
scanners = parse_data(input_text)
master = scanners[0]
minion = scanners[1]
while any([not scanner.aligned for scanner in scanners.values()]):
    for scanner in scanners.values():
        if scanner.aligned:
            continue
        for other in scanners.values():
            if not other.aligned:
                continue
            print(f"Trying to align: {scanner.scanner_id} with {other.scanner_id}")
            scanner.align(other)
            if scanner.aligned:
                break

print(f"Explore minion")
for point in minion.beacons:
    for other in master.beacons:
        for rotation in generate_all_rotations(point):
            if calculate_translation_vector(rotation, other) ==  minion.translation_vector:
                print(f"minion: {point}, rotation: {rotation} origin: {other}, translation_vector: {minion.translation_vector}")
# for point in minion.beacons:
#     print(point)
#     if any(abs(val)==528 for val in point):
#         print(f"beacon: {point}")
#         t_point = []
#         for idx, val in enumerate(point):
#             t_point.append(val + minion.translation_vector[idx])
#         print(f"translated: {t_point}")

for scanner in scanners.values():
    scanner.translate()
    print(scanner)
    print(scanner.beacons.intersection(master.beacons))


In [ ]:
print(f"Explore minion")
for point in minion.beacons:
    for other in master.beacons:
        for rotation in generate_all_rotations(point):
            print(f"minion: {point}, rotation: {rotation} origin: {other}, translation_vector: {minion.translation_vector}, calculated: {calculate_translation_vector(rotation, other)}")

In [ ]:
calculate_translation_vector((355, 545, 477),(-355, 545, -477))

In [ ]:
import numpy as np
from itertools import permutations, product

def generate_all_rotations(point):
    """
    Generate all 24 unique orientations of a 3D point using the cube's full symmetry group.

    Parameters:
        point (tuple): The initial 3D point (x, y, z).

    Returns:
        set: A set of 24 unique rotated points.
    """
    point_vector = np.array(point)
    unique_points = set()

    # Generate the 24 rotation matrices using cube symmetries
    base_axes = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # Standard basis
    signs = [1, -1]

    for permuted_axes in permutations(base_axes):
        for sign_comb in product(signs, repeat=3):
            rotation_matrix = np.array([s * np.array(a) for s, a in zip(sign_comb, permuted_axes)])
            
            # Ensure the rotation matrix is valid (determinant == 1)
            if np.linalg.det(rotation_matrix) == 1:
                rotated_point = tuple(np.dot(rotation_matrix, point_vector))
                unique_points.add(rotated_point)

    return unique_points


# Test the function
point = (8, 0, 7)
rotated_points = generate_all_rotations(point)
print(f"Found {len(rotated_points)} unique rotations.")
print(rotated_points)


In [ ]:
a = set([1,2])
b = set(a)
a = set([3,4])
print(a, b)

In [ ]:
import numpy as np
from itertools import permutations, product

def rotate_point(point, rotation_matrix):
    return tuple(np.dot(rotation_matrix, point))

def generate_rotation_matrices():
    """Generate all 24 valid rotation matrices for 90-degree cube rotations."""
    base_axes = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
    signs = [1, -1]
    matrices = []
    for permuted_axes in permutations(base_axes):
        for sign_comb in product(signs, repeat=3):
            matrix = np.array([s * np.array(axis) for s, axis in zip(sign_comb, permuted_axes)])
            if np.linalg.det(matrix) == 1:
                matrices.append(matrix)
    return matrices

def find_matching_points(scanner_0_points, scanner_1_points):
    """Find 12 matching points under any rotation."""
    rotation_matrices = generate_rotation_matrices()
    
    for rotation_matrix in rotation_matrices:
        rotated_scanner_1 = [rotate_point(p, rotation_matrix) for p in scanner_1_points]
        
        # Compare each rotated beacon set
        for p0 in scanner_0_points:
            for p1 in rotated_scanner_1:
                translation = np.subtract(p0, p1)
                transformed_beacons = {tuple(np.add(beacon, translation)) for beacon in rotated_scanner_1}
                
                # Check if there are at least 12 matching points
                common_beacons = transformed_beacons.intersection(scanner_0_points)
                if len(common_beacons) >= 12:
                    return rotation_matrix, translation, common_beacons
    return None, None, None

def transform_all_points(scanner_1_points, rotation_matrix, translation_vector):
    """Transform all scanner 1 points relative to scanner 0."""
    rotated_points = [rotate_point(p, rotation_matrix) for p in scanner_1_points]
    transformed_points = [tuple(np.add(p, translation_vector)) for p in rotated_points]
    return transformed_points

# Data from the scanners
scanner_0_points = [
    (404, -588, -901), (528, -643, 409), (-838, 591, 734), (390, -675, -793),
    (-537, -823, -458), (-485, -357, 347), (-345, -311, 381), (-661, -816, -575),
    (-876, 649, 763), (-618, -824, -621), (553, 345, -567), (474, 580, 667),
    (-447, -329, 318), (-584, 868, -557), (544, -627, -890), (564, 392, -477),
    (455, 729, 728), (-892, 524, 684), (-689, 845, -530), (423, -701, 434),
    (7, -33, -71), (630, 319, -379), (443, 580, 662), (-789, 900, -551),
    (459, -707, 401)
]

scanner_1_points = [
    (686, 422, 578), (605, 423, 415), (515, 917, -361), (-336, 658, 858),
    (95, 138, 22), (-476, 619, 847), (-340, -569, -846), (567, -361, 727),
    (-460, 603, -452), (669, -402, 600), (729, 430, 532), (-500, -761, 534),
    (-322, 571, 750), (-466, -666, -811), (-429, -592, 574), (-355, 545, -477),
    (703, -491, -529), (-328, -685, 520), (413, 935, -424), (-391, 539, -444),
    (586, -435, 557), (-364, -763, -893), (807, -499, -711), (755, -354, -619),
    (553, 889, -390)
]

# Step 1: Find matching points
rotation_matrix, translation_vector, matched_points = find_matching_points(scanner_0_points, scanner_1_points)

if rotation_matrix is not None:
    print(f"Rotation Matrix:\n{rotation_matrix}")
    print(f"Translation Vector: {translation_vector}")
    print(f"Matched Points (12+): {matched_points}")
    
    # Step 2: Transform all points
    transformed_points = transform_all_points(scanner_1_points, rotation_matrix, translation_vector)
    print(f"Transformed Scanner 1 Points Relative to Scanner 0: {transformed_points}")
    intersection = set(scanner_0_points).intersection(set(transformed_points))
    print(f"intersection: {len(intersection)} {intersection}")
else:
    print("Failed to find alignment.")


Rotation Matrix:
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Translation Vector: [   68 -1246   -43]
Matched Points (12+): {(390, -675, -793), (-345, -311, 381), (404, -588, -901), (459, -707, 401), (-661, -816, -575), (544, -627, -890), (528, -643, 409), (-485, -357, 347), (-537, -823, -458), (423, -701, 434), (-618, -824, -621), (-447, -329, 318)}
Transformed Scanner 1 Points Relative to Scanner 0: [(-618, -824, -621), (-537, -823, -458), (-447, -329, 318), (404, -588, -901), (-27, -1108, -65), (544, -627, -890), (408, -1815, 803), (-499, -1607, -770), (528, -643, 409), (-601, -1648, -643), (-661, -816, -575), (568, -2007, -577), (390, -675, -793), (534, -1912, 768), (497, -1838, -617), (423, -701, 434), (-635, -1737, 486), (396, -1931, -563), (-345, -311, 381), (459, -707, 401), (-518, -1681, -600), (432, -2009, 850), (-739, -1745, 668), (-687, -1600, 576), (-485, -357, 347)]
intersection: 12 {(390, -675, -793), (-345, -311, 381), (404, -588, -901), (459, -707, 401), (-661, -816, -575), (5

In [ ]:
import numpy as np
from itertools import permutations, product

def rotate_point(point, rotation_matrix):
    return tuple(np.dot(rotation_matrix, point))

def generate_rotation_matrices():
    """Generate all 24 valid rotation matrices for 90-degree cube rotations."""
    base_axes = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
    signs = [1, -1]
    matrices = []
    for permuted_axes in permutations(base_axes):
        for sign_comb in product(signs, repeat=3):
            matrix = np.array([s * np.array(axis) for s, axis in zip(sign_comb, permuted_axes)])
            if np.linalg.det(matrix) == 1:
                matrices.append(matrix)
    return matrices

def find_matching_points(scanner_0_points, scanner_1_points):
    """Find 12 matching points under any rotation."""
    rotation_matrices = generate_rotation_matrices()
    
    for rotation_matrix in rotation_matrices:
        rotated_scanner_1 = [rotate_point(p, rotation_matrix) for p in scanner_1_points]
        
        # Compare each rotated beacon set
        for p0 in scanner_0_points:
            for p1 in rotated_scanner_1:
                translation = np.subtract(p0, p1)
                transformed_beacons = {tuple(np.add(beacon, translation)) for beacon in rotated_scanner_1}
                
                # Check if there are at least 12 matching points
                common_beacons = transformed_beacons.intersection(scanner_0_points)
                if len(common_beacons) >= 12:
                    return rotation_matrix, translation, common_beacons
    return None, None, None

def transform_all_points(scanner_1_points, rotation_matrix, translation_vector):
    """Transform all scanner 1 points relative to scanner 0."""
    rotated_points = [rotate_point(p, rotation_matrix) for p in scanner_1_points]
    transformed_points = [tuple(np.add(p, translation_vector)) for p in rotated_points]
    return transformed_points

def calculate_scanner_location(translation_vector):
    """Find scanner 1's position relative to scanner 0."""
    return tuple(-np.array(translation_vector))

# Data from the scanners
scanner_0_points = [
    (404, -588, -901), (528, -643, 409), (-838, 591, 734), (390, -675, -793),
    (-537, -823, -458), (-485, -357, 347), (-345, -311, 381), (-661, -816, -575),
    (-876, 649, 763), (-618, -824, -621), (553, 345, -567), (474, 580, 667),
    (-447, -329, 318), (-584, 868, -557), (544, -627, -890), (564, 392, -477),
    (455, 729, 728), (-892, 524, 684), (-689, 845, -530), (423, -701, 434),
    (7, -33, -71), (630, 319, -379), (443, 580, 662), (-789, 900, -551),
    (459, -707, 401)
]

scanner_1_points = [
    (686, 422, 578), (605, 423, 415), (515, 917, -361), (-336, 658, 858),
    (95, 138, 22), (-476, 619, 847), (-340, -569, -846), (567, -361, 727),
    (-460, 603, -452), (669, -402, 600), (729, 430, 532), (-500, -761, 534),
    (-322, 571, 750), (-466, -666, -811), (-429, -592, 574), (-355, 545, -477),
    (703, -491, -529), (-328, -685, 520), (413, 935, -424), (-391, 539, -444),
    (586, -435, 557), (-364, -763, -893), (807, -499, -711), (755, -354, -619),
    (553, 889, -390)
]

# Step 1: Find matching points
rotation_matrix, translation_vector, matched_points = find_matching_points(scanner_0_points, scanner_1_points)

if rotation_matrix is not None:
    print(f"Rotation Matrix:\n{rotation_matrix}")
    print(f"Translation Vector: {translation_vector}")
    
    # Step 2: Calculate Scanner 1's Location
    scanner_1_location = calculate_scanner_location(translation_vector)
    print(f"Scanner 1's Location Relative to Scanner 0: {scanner_1_location}")
    
    # Step 3: Transform all points
    transformed_points = transform_all_points(scanner_1_points, rotation_matrix, translation_vector)
    print(f"Transformed Scanner 1 Points Relative to Scanner 0: {transformed_points}")
else:
    print("Failed to find alignment.")


Rotation Matrix:
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Translation Vector: [   68 -1246   -43]
Scanner 1's Location Relative to Scanner 0: (-68, 1246, 43)
Transformed Scanner 1 Points Relative to Scanner 0: [(-618, -824, -621), (-537, -823, -458), (-447, -329, 318), (404, -588, -901), (-27, -1108, -65), (544, -627, -890), (408, -1815, 803), (-499, -1607, -770), (528, -643, 409), (-601, -1648, -643), (-661, -816, -575), (568, -2007, -577), (390, -675, -793), (534, -1912, 768), (497, -1838, -617), (423, -701, 434), (-635, -1737, 486), (396, -1931, -563), (-345, -311, 381), (459, -707, 401), (-518, -1681, -600), (432, -2009, 850), (-739, -1745, 668), (-687, -1600, 576), (-485, -357, 347)]


In [ ]:
rotation_matrix, translation_vector, matched_points = find_matching_points(scanners[3].beacons, scanners[1].beacons)
print(rotation_matrix, translation_vector, matched_points)

[[1 0 0]
 [0 1 0]
 [0 0 1]] [-388 1022   89] {(-660, 1699, -800), (266, 1833, 603), (349, 1866, 646), (435, 1914, -573), (383, 1777, -483), (-786, 1602, -765), (247, 1907, 773), (-749, 1676, 620), (-820, 1507, 580), (-648, 1583, 566), (487, 1769, -665), (-684, 1505, -847)}


None None None


In [ ]:
a = set([1,2,3])
a.update([4,5,6])
print(a)

{1, 2, 3, 4, 5, 6}
